In [ ]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


from bs4 import BeautifulSoup

# 高雄市-區-里
url = 'https://zh.wikipedia.org/wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E6%9D%91%E9%87%8C%E5%88%97%E8%A1%A8#%E9%AB%98%E9%9B%84%E5%B8%82'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}

source = requests.get(url, headers=headers).text

soup = BeautifulSoup(source, 'html5lib')
#print(soup.prettify())

tables = soup.find_all('table')
"""
for i_t in range(len(tables)):
    if '清豐里' in tables[i_t].text:
        print(i_t)
        print(tables[i_t].text)
"""
        
import re

rows = tables[18].find_all('tr') + tables[19].find_all('tr')
dict_dn_kh = {}

cur_d = None
for i_r in range(len(rows)):
    #print(i_r, '----------\n')
    #print(rows[i_r])
    cells = rows[i_r].find_all('td')
    for i_c in range(len(cells)):
        #print('  ', i_c, '============\n')
        #print('  ', cells[i_c].text)
        for each_line in cells[i_c].text.split('\n'):
            if '' in dict_dn_kh.keys():
                ppppp
            print('cur_d: [%s]' % cur_d)
            if each_line == '':
                continue
            print('each_line:', each_line)
            if '：' in each_line:
                if len(each_line) > 0 and '（續）' not in each_line:
                    cur_d = each_line.strip().replace('：', '')
                    print('new cur_d:', cur_d)
                    if cur_d != '':
                        dict_dn_kh[cur_d] = []
            else:
                dict_dn_kh[cur_d].append(re.sub('\（.+\）', '', each_line.strip()))
            #neighbors = [e for e in splited[1].split('\n') if len(e) > 0]
            #print('district:', district)
            #dict_dn_kh[district] = neighbors
print('dict_dn_kh:', dict_dn_kh)

#address = '台北市, 台灣'
address = '高雄市, 台灣'
geolocator = Nominatim(user_agent="to_explorer")

location = geolocator.geocode(address)
latitude_center = location.latitude
longitude_center = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude_center, longitude_center))
map_kh = folium.Map(location=[latitude_center,longitude_center],zoom_start=13)

count_d = 0
count_n = 0
for d, ns in dict_dn_kh.items():
    count_d += 1
    count_n += len(ns)
print('we have [%d] districts and [%d] neighborhoods' % (count_d, count_n))

from tqdm import tqdm

location_districts = {'Districts':[], 'Neighborhood':[], 'x':[], 'y':[]}
for district, neighborhoods in tqdm(dict_dn_kh.items()):
    print('handling:', district)
    for n in neighborhoods:
        addr_temp = '%s, %s, %s' % (n, district, address)
        location = geolocator.geocode(addr_temp)
        if location:
            x = location.latitude
            y = location.longitude
            location_districts['Districts'].append(district)
            location_districts['Neighborhood'].append(n)
            location_districts['x'].append(x)
            location_districts['y'].append(y)
            #print(addr_temp, x, y)
            label = '高雄市{}{}'.format(district, n)
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
            [x, y],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_kh)
        else:
            print('Cannot find location of:', addr_temp)
df_location_districts = pd.DataFrame(location_districts)
print(df_location_districts.head())
map_kh

CLIENT_ID = 'CYRKJGO1UOMYCBDU5EUJKMYXPDGS0SKRHAKAYPG4UTM4SOOC' # your Foursquare ID
CLIENT_SECRET = 'EHQ5XZMCNN2ELZJN5RCEFMC5LOKEEJUCIDKCAY22FZSRC0T2' # your Foursquare Secret
ACCESS_TOKEN = 'EKQFKVI01XTEHXFUTUHURDWV5N31J5KBFJQ1YNQEDVD1GF1W'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

def getNearbyVenues(names_d, names_n, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name_d, name_n, lat, lng in tqdm(zip(names_d, names_n, latitudes, longitudes)):
        name = name_d + name_n
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #print(url)
            
        # make the GET request
        #print(requests.get(url).json()["response"])
        #if 'groups' not in requests.get(url).json()["response"]:
        #    print(requests.get(url).json()["response"])
        #    continue
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #print('results:', results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

kh_venues = getNearbyVenues(names_d=df_location_districts['Districts'], names_n=df_location_districts['Neighborhood'],
                                   latitudes=df_location_districts['x'],
                                   longitudes=df_location_districts['y']
                                  )
print(kh_venues.head())

In [1]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
!pip install bs4

from bs4 import BeautifulSoup
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 4.0 MB/s  eta 0:00:01
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filenam

In [21]:

# 台北市
#url_district = 'https://www.astrocode.net/%E5%8F%B0%E7%81%A3%E5%90%84%E7%B8%A3%E5%B8%82%E5%9C%B0%E5%8D%80%E7%B6%93%E7%B7%AF%E5%BA%A6/'
# 這個不對
url_district = 'https://en.wikipedia.org/wiki/Taipei'
url_district = 'https://zh.wikipedia.org/wiki/%E8%87%BA%E5%8C%97%E5%B8%82%E8%A1%8C%E6%94%BF%E5%8D%80%E5%8A%83'
url_district = 'https://zh.wikipedia.org/wiki/%E9%AB%98%E9%9B%84%E5%B8%82%E8%A1%8C%E6%94%BF%E5%8D%80%E5%8A%83'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}

source = requests.get(url_district, headers=headers).text

soup = BeautifulSoup(source, 'html5lib')
print(soup.prettify())

district_location = {'Districts':[], 'x':[], 'y':[]}
tables = soup.find_all("table",{"class":"wikitable"})

districts = []
for row in tables[-2].find_all('tr'):
    #print('row:\n', row)
    #print('-'*30)
    cells = row.find_all('td')
    #if len(cells) > 0 and 'rowspan' in str(cells[0]):
    if len(cells) > 0 and '區' in cells[0].text:
        districts.append(cells[0].text.strip())
        #print('cells[0]:\n', cells[0].text)
        #if 'rowspan' in cells[0]:
        #    print('cells[0]:', cells.text)
    #for i in range(len(cells)):
    #    print('----[%d] cells:\n[%s]' % (i, cells[i].text))
    #if len(cells) > 0:
    #    print('----\n', cells[0].text)
print('districts:', districts)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="zh">
 <head>
  <meta charset="utf-8"/>
  <title>
   高雄市行政區劃 - 维基百科，自由的百科全书
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"zh","wgMonthNames":["","1月","2月","3月","4月","5月","6月","7月","8月","9月","10月","11月","12月"],"wgRequestId":"4f45a525-4c28-47a2-8ce3-a26a638a6857","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"高雄市行政區劃","wgTitle":"高雄市行政區劃","wgCurRevisionId":64277267,"wgRevisionId":64277267,"wgArticleId":417202,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["高雄市行政區劃"],"wgPageContentLanguage":"zh","wgPageContentModel":"wikitext","wgRelevantPageName":"高雄市行政區劃","wgRelevantArticleId":417202,"wgUserVariant":"zh","wgIsProbablyEditable":!0,"wgRelevantPageIsProbablyEditable":!0,"

IndexError: list index out of range

In [77]:
# 高雄市-區-里
url = 'https://zh.wikipedia.org/wiki/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E6%9D%91%E9%87%8C%E5%88%97%E8%A1%A8#%E9%AB%98%E9%9B%84%E5%B8%82'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}

source = requests.get(url, headers=headers).text

soup = BeautifulSoup(source, 'html5lib')
#print(soup.prettify())

tables = soup.find_all('table')
"""
for i_t in range(len(tables)):
    if '清豐里' in tables[i_t].text:
        print(i_t)
        print(tables[i_t].text)
"""
        
import re

rows = tables[18].find_all('tr') + tables[19].find_all('tr')
dict_dn_kh = {}

cur_d = None
for i_r in range(len(rows)):
    #print(i_r, '----------\n')
    #print(rows[i_r])
    cells = rows[i_r].find_all('td')
    for i_c in range(len(cells)):
        #print('  ', i_c, '============\n')
        #print('  ', cells[i_c].text)
        for each_line in cells[i_c].text.split('\n'):
            if '' in dict_dn_kh.keys():
                ppppp
            print('cur_d: [%s]' % cur_d)
            if each_line == '':
                continue
            print('each_line:', each_line)
            if '：' in each_line:
                if len(each_line) > 0 and '（續）' not in each_line:
                    cur_d = each_line.strip().replace('：', '')
                    print('new cur_d:', cur_d)
                    if cur_d != '':
                        dict_dn_kh[cur_d] = []
            else:
                dict_dn_kh[cur_d].append(re.sub('\（.+\）', '', each_line.strip()))
            #neighbors = [e for e in splited[1].split('\n') if len(e) > 0]
            #print('district:', district)
            #dict_dn_kh[district] = neighbors
print('dict_dn_kh:', dict_dn_kh)

cur_d: [None]
cur_d: [None]
cur_d: [None]
cur_d: [None]
cur_d: [None]
cur_d: [None]
cur_d: [None]
cur_d: [None]
each_line: 楠梓區：
new cur_d: 楠梓區
cur_d: [楠梓區]
each_line: 清豐里
cur_d: [楠梓區]
each_line: 東寧里
cur_d: [楠梓區]
each_line: 五常里
cur_d: [楠梓區]
each_line: 享平里
cur_d: [楠梓區]
each_line: 中陽里
cur_d: [楠梓區]
each_line: 惠楠里
cur_d: [楠梓區]
each_line: 惠民里
cur_d: [楠梓區]
each_line: 惠豐里
cur_d: [楠梓區]
each_line: 錦屏里
cur_d: [楠梓區]
each_line: 玉屏里
cur_d: [楠梓區]
each_line: 金田里
cur_d: [楠梓區]
each_line: 稔田里
cur_d: [楠梓區]
each_line: 瑞屏里
cur_d: [楠梓區]
each_line: 翠屏里
cur_d: [楠梓區]
each_line: 宏南里
cur_d: [楠梓區]
each_line: 宏毅里
cur_d: [楠梓區]
each_line: 宏榮里
cur_d: [楠梓區]
each_line: 廣昌里
cur_d: [楠梓區]
each_line: 久昌里
cur_d: [楠梓區]
each_line: 大昌里
cur_d: [楠梓區]
each_line: 福昌里
cur_d: [楠梓區]
each_line: 盛昌里
cur_d: [楠梓區]
each_line: 泰昌里
cur_d: [楠梓區]
each_line: 興昌里
cur_d: [楠梓區]
each_line: 建昌里
cur_d: [楠梓區]
each_line: 宏昌里
cur_d: [楠梓區]
each_line: 和昌里
cur_d: [楠梓區]
each_line: 慶昌里
cur_d: [楠梓區]
each_line: 隆昌里
cur_d: [楠梓區]
each_line: 秀昌里
cur_d: [楠梓區]
each

each_line: 田尾里
cur_d: [湖內區]
each_line: 劉家里
cur_d: [湖內區]
each_line: 海埔里
cur_d: [湖內區]
each_line: 海山里
cur_d: [湖內區]
each_line: 公館里
cur_d: [湖內區]
each_line: 太爺里
cur_d: [湖內區]
each_line: 葉厝里
cur_d: [湖內區]
each_line: 中賢里
cur_d: [湖內區]
each_line: 逸賢里
cur_d: [湖內區]
each_line: 文賢里
cur_d: [湖內區]
each_line: 忠興里
cur_d: [湖內區]
each_line: 彌陀區：
new cur_d: 彌陀區
cur_d: [彌陀區]
each_line: 文安里
cur_d: [彌陀區]
each_line: 光和里
cur_d: [彌陀區]
each_line: 南寮里
cur_d: [彌陀區]
each_line: 海尾里
cur_d: [彌陀區]
each_line: 過港里
cur_d: [彌陀區]
cur_d: [彌陀區]
cur_d: [彌陀區]
each_line: 彌陀區（續）：
cur_d: [彌陀區]
each_line: 漯底里
cur_d: [彌陀區]
each_line: 彌仁里
cur_d: [彌陀區]
each_line: 彌陀里
cur_d: [彌陀區]
each_line: 彌靖里
cur_d: [彌陀區]
each_line: 彌壽里
cur_d: [彌陀區]
each_line: 舊港里
cur_d: [彌陀區]
each_line: 鹽埕里
cur_d: [彌陀區]
each_line: 內門區：
new cur_d: 內門區
cur_d: [內門區]
each_line: 三平里
cur_d: [內門區]
each_line: 中埔里
cur_d: [內門區]
each_line: 永吉里
cur_d: [內門區]
each_line: 永富里
cur_d: [內門區]
each_line: 永興里
cur_d: [內門區]
each_line: 石坑里
cur_d: [內門區]
each_line: 光興里
cur_d: [內門區]
each_line: 東埔里

In [61]:
"""
# 高雄市/區
districts = set()
tables = soup.find_all("table")
#print(tables[-2].prettify())

rows = tables[-2].find_all('tr')
for i_row in range(4, len(rows)):
    #print('row:\n', row)
    #print('-'*30)
    cells = rows[i_row].find_all('td')
    #for i in range(len(cells)):
    #    print('cells[%d]:\n[%s]' % (i, cells[i]))
    if len(cells) > 0:
        #print(cells[0])
        lis = cells[0].find_all('li')
        for i_li in range(len(lis)):
            #print('rows[%d]cells[0]lis[%d]:\n[%s]' % (i_row, i_li, lis[i_li]))
            #print('-'*30)
            if '區' in lis[i_li].text and len(lis[i_li].text) <= 4:
                districts.add(lis[i_li].text.strip())
print('districts:', districts)
"""
print(len(dict_dn_kh))
print(dict_dn_kh.keys())

count_d = 0
count_n = 0
for d, ns in dict_dn_kh.items():
    count_d += 1
    count += len(ns)
print('we have [%d] districts and [%d] neighborhoods' % (count_d, count_n))

38
dict_keys(['楠梓區', '左營區', '三民區', '新興區', '前金區', '鼓山區', '苓雅區', '鹽埕區', '前鎮區', '旗津區', '小港區', '鳳山區', '岡山區', '旗山區', '美濃區', '大寮區', '林園區', '仁武區', '路竹區', '大樹區', '鳥松區', '梓官區', '橋頭區', '茄萣區', '大社區', '燕巢區', '阿蓮區', '湖內區', '彌陀區', '內門區', '六龜區', '永安區', '杉林區', '田寮區', '甲仙區', '桃源區', '茂林區', '那瑪夏區'])


In [78]:
#address = '台北市, 台灣'
address = '高雄市, 台灣'
geolocator = Nominatim(user_agent="to_explorer")

location = geolocator.geocode(address)
latitude_center = location.latitude
longitude_center = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude_center, longitude_center))
map_kh = folium.Map(location=[latitude_center,longitude_center],zoom_start=13)

count_d = 0
count_n = 0
for d, ns in dict_dn_kh.items():
    count_d += 1
    count_n += len(ns)
print('we have [%d] districts and [%d] neighborhoods' % (count_d, count_n))

from tqdm import tqdm

location_districts = {'Districts':[], 'Neighborhood':[], 'x':[], 'y':[]}
for district, neighborhoods in tqdm(dict_dn_kh.items()):
    print('handling:', district)
    for n in neighborhoods:
        addr_temp = '%s, %s, %s' % (n, district, address)
        location = geolocator.geocode(addr_temp)
        if location:
            x = location.latitude
            y = location.longitude
            location_districts['Districts'].append(district)
            location_districts['Neighborhood'].append(n)
            location_districts['x'].append(x)
            location_districts['y'].append(y)
            #print(addr_temp, x, y)
            label = '高雄市{}{}'.format(district, n)
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
            [x, y],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_kh)
        else:
            print('Cannot find location of:', addr_temp)
df_location_districts = pd.DataFrame(location_districts)
print(df_location_districts.head())
map_kh

  0%|          | 0/38 [00:00<?, ?it/s]

The geograpical coordinate of 高雄市, 台灣 are 22.6203348, 120.3120375.
we have [38] districts and [856] neighborhoods
handling: 楠梓區


  3%|▎         | 1/38 [00:18<11:24, 18.51s/it]

handling: 左營區
Cannot find location of: 復興里, 左營區, 高雄市, 台灣


  5%|▌         | 2/38 [00:38<11:37, 19.38s/it]

...

handling: 茂林區


 97%|█████████▋| 37/38 [07:09<00:03,  3.44s/it]

handling: 那瑪夏區


100%|██████████| 38/38 [07:11<00:00, 11.35s/it]


  Districts Neighborhood          x           y
0       楠梓區          清豐里  22.745355  120.329905
1       楠梓區          東寧里  22.733572  120.329930
2       楠梓區          五常里  22.728447  120.333994
3       楠梓區          享平里  22.729450  120.329497
4       楠梓區          中陽里  22.722662  120.327786


In [29]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_990a7e4b614046be8a53265290fc9cab = 'https://s3.ap-geo.objectstorage.softlayer.net'
else:
    endpoint_990a7e4b614046be8a53265290fc9cab = 'https://s3.ap-geo.objectstorage.service.networklayer.com'

client_990a7e4b614046be8a53265290fc9cab = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='TRxbSWfQmx-ZTNpBHWwbZdlCXaHnlFECGoCleCY8USY5',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_990a7e4b614046be8a53265290fc9cab)

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_2 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-162f2eaf-eef3-4ec0-94df-77cc5353a9da',
    'IBM_API_KEY_ID': 'TRxbSWfQmx-ZTNpBHWwbZdlCXaHnlFECGoCleCY8USY5',
    'ENDPOINT': 'https://s3.ap-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'courseracapstone-donotdelete-pr-nqeyo6hfku3v0b',
    'FILE': 'IncomeTaxByNeighborhoods_2017.csv'
}
body = client_990a7e4b614046be8a53265290fc9cab.get_object(Bucket='courseracapstone-donotdelete-pr-nqeyo6hfku3v0b',Key='IncomeTaxByNeighborhoods_2017.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_income_all = pd.read_csv(body)
df_income_all.reset_index(inplace=True)
#print(df_income_all.head())
df_income = df_income_all[df_income_all['﻿縣市'] == '高雄市']
#print(df_income.head())

selected_districts = []

df_income_district = df_income.groupby('鄉鎮市區').mean().reset_index()
selected_districts += df_income_district.sort_values(by='中位數', ascending=False).head(5)['鄉鎮市區'].to_list()
selected_districts += df_income_district.sort_values(by='平均數', ascending=False).head(5)['鄉鎮市區'].to_list()
selected_districts = set([e for e in set(selected_districts) if '區' in e])
print('selected_districts:', selected_districts)


selected_districts: {'苓雅區', '梓官區', '新興區', '前金區', '鳥松區', '楠梓區', '大寮區'}


In [21]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_990a7e4b614046be8a53265290fc9cab = 'https://s3.ap-geo.objectstorage.softlayer.net'
else:
    endpoint_990a7e4b614046be8a53265290fc9cab = 'https://s3.ap-geo.objectstorage.service.networklayer.com'

client_990a7e4b614046be8a53265290fc9cab = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='TRxbSWfQmx-ZTNpBHWwbZdlCXaHnlFECGoCleCY8USY5',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_990a7e4b614046be8a53265290fc9cab)

body = client_990a7e4b614046be8a53265290fc9cab.get_object(Bucket='courseracapstone-donotdelete-pr-nqeyo6hfku3v0b',Key='DistrictNeighborhoodVenue_KH_v1.1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

kh_venues = pd.read_csv(body)
kh_venues.head()


,Unnamed: 0,District,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,楠梓區,清豐里,22.745355,120.329905,大眾羊肉,22.749278,120.330398,Asian Restaurant
1,1,楠梓區,東寧里,22.733572,120.329930,台糖量販楠梓店,22.736173,120.331014,Shopping Mall
2,2,楠梓區,東寧里,22.733572,120.329930,smallの萱,22.731524,120.327888,Restaurant
3,3,楠梓區,東寧里,22.733572,120.329930,廣東汕頭自製雞蛋麵,22.736191,120.332240,Soup Place
4,4,楠梓區,東寧里,22.733572,120.329930,85度C,22.729231,120.329793,Coffee Shop


In [79]:
CLIENT_ID = 'CYRKJGO1UOMYCBDU5EUJKMYXPDGS0SKRHAKAYPG4UTM4SOOC' # your Foursquare ID
CLIENT_SECRET = 'EHQ5XZMCNN2ELZJN5RCEFMC5LOKEEJUCIDKCAY22FZSRC0T2' # your Foursquare Secret
ACCESS_TOKEN = 'EKQFKVI01XTEHXFUTUHURDWV5N31J5KBFJQ1YNQEDVD1GF1W'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CYRKJGO1UOMYCBDU5EUJKMYXPDGS0SKRHAKAYPG4UTM4SOOC
CLIENT_SECRET:EHQ5XZMCNN2ELZJN5RCEFMC5LOKEEJUCIDKCAY22FZSRC0T2


In [84]:
def getNearbyVenues(names_d, names_n, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name_d, name_n, lat, lng in tqdm(zip(names_d, names_n, latitudes, longitudes)):
        name = name_d + name_n
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #print(url)
            
        # make the GET request
        #print(requests.get(url).json()["response"])
        #if 'groups' not in requests.get(url).json()["response"]:
        #    print(requests.get(url).json()["response"])
        #    continue
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #print('results:', results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
IfCallFourSquare = False
if IfCallFourSquare == True:
    print('Calling foursquare API...')
    kh_venues = getNearbyVenues(names_d=df_location_districts['Districts'], names_n=df_location_districts['Neighborhood'],
                                       latitudes=df_location_districts['x'],
                                       longitudes=df_location_districts['y']
                                      )
    print(kh_venues.head())
else:
    print('Dont call API!')
    

Dont call API!


In [30]:
ii_selected = []
for e in kh_venues.index:
    if kh_venues.loc[e, 'District'] in selected_districts:
        ii_selected.append(e)
kh_venues_selected = kh_venues.loc[ii_selected]
kh_venues_selected

,Unnamed: 0,District,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,楠梓區,清豐里,22.745355,120.329905,大眾羊肉,22.749278,120.330398,Asian Restaurant
1,1,楠梓區,東寧里,22.733572,120.329930,台糖量販楠梓店,22.736173,120.331014,Shopping Mall
2,2,楠梓區,東寧里,22.733572,120.329930,smallの萱,22.731524,120.327888,Restaurant
3,3,楠梓區,東寧里,22.733572,120.329930,廣東汕頭自製雞蛋麵,22.736191,120.332240,Soup Place
4,4,楠梓區,東寧里,22.733572,120.329930,85度C,22.729231,120.329793,Coffee Shop
...,...,...,...,...,...,...,...,...,...
4252,4252,梓官區,智蚵里,22.728765,120.260566,珍饌活海產,22.728035,120.257248,Seafood Restaurant
4253,4253,梓官區,禮蚵里,22.732499,120.250271,屏東涼拌滷味,22.731925,120.248422,Snack Place
4254,4254,梓官區,禮蚵里,22.732499,120.250271,五金咖啡,22.734003,120.252072,Coffee Shop
4255,4255,梓官區,禮蚵里,22.732499,120.250271,德記無骨香酥雞•薯條,22.733036,120.247834,Fast Food Restaurant


In [31]:
# one hot encoding
kh_onehot = pd.get_dummies(kh_venues_selected[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kh_onehot['DN'] = kh_venues_selected['District'] + kh_venues_selected['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kh_onehot.columns[-1]] + list(kh_onehot.columns[:-1])
kh_onehot = kh_onehot[fixed_columns]

print(kh_onehot.head())

       DN  American Restaurant  Argentinian Restaurant  Asian Restaurant  \
0  楠梓區清豐里                    0                       0                 1   
1  楠梓區東寧里                    0                       0                 0   
2  楠梓區東寧里                    0                       0                 0   
3  楠梓區東寧里                    0                       0                 0   
4  楠梓區東寧里                    0                       0                 0   

   BBQ Joint  Bagel Shop  Bakery  Baseball Stadium  Basketball Court  \
0          0           0       0                 0                 0   
1          0           0       0                 0                 0   
2          0           0       0                 0                 0   
3          0           0       0                 0                 0   
4          0           0       0                 0                 0   

   Bed & Breakfast  ...  Sushi Restaurant  Taiwanese Restaurant  Tea Room  \
0                0  ...          

In [45]:
kh_grouped = kh_onehot.groupby('DN').mean().reset_index()
num_top_venues = 10

n_HasGymMetro = []

for hood in kh_grouped['DN']:
    HasGym = False
    HasMetro = False
    #print("----"+hood+"----")
    #continue
    temp = kh_grouped[kh_grouped['DN'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    temp = temp.sort_values('freq', ascending=False).reset_index(drop=True)
    for ii in range(len(temp.index)):
        if HasMetro and HasGym:
            n_HasGymMetro.append(hood)
            break
        else:
            if HasMetro == False:
                if temp.loc[ii, 'venue'] == 'Metro Station':
                    HasMetro = True
            if HasGym == False:
                if 'gym' in temp.loc[ii, 'venue'].lower():
                    #print('hit', temp.loc[ii, 'venue'])
                    HasGym = True
        
        #if temp.loc[ii, 'freq'] > 0:
        #    print('%s(%f)' % (temp.loc[ii, 'venue'], temp.loc[ii, 'freq']))
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')
    #break
    
print('n_HasGymMetro:', n_HasGymMetro)

n_HasGymMetro: ['前金區三川里', '前金區北金里', '前金區博孝里', '前金區國民里', '前金區後金里', '前金區復元里', '前金區文東里', '前金區文西里', '前金區新生里', '前金區東金里', '前金區林投里', '前金區榮復里', '前金區民生里', '前金區社東里', '前金區社西里', '前金區草江里', '前金區長城里', '前金區長生里', '前金區長興里', '前金區青山里', '大寮區中興里', '大寮區前庄里', '大寮區大寮里', '大寮區昭明里', '大寮區會社里', '新興區中東里', '新興區仁聲里', '新興區光耀里', '新興區南港里', '新興區大明里', '新興區建興里', '新興區建華里', '新興區德政里', '新興區德望里', '新興區德生里', '新興區愛平里', '新興區成功里', '新興區振成里', '新興區振華里', '新興區文昌里', '新興區新江里', '新興區明莊里', '新興區東坡里', '新興區榮治里', '新興區正氣里', '新興區永寧里', '新興區浩然里', '新興區漢民里', '新興區玉衡里', '新興區秋山里', '新興區興昌里', '新興區華聲里', '新興區蕉園里', '新興區長驛里', '新興區開平里', '新興區順昌里', '新興區黎明里', '梓官區中崙里', '梓官區信蚵里', '梓官區大舍里', '梓官區智蚵里', '梓官區禮蚵里', '梓官區茄苳里', '梓官區赤西里', '楠梓區中興里', '楠梓區中陽里', '楠梓區久昌里', '楠梓區五常里', '楠梓區享平里', '楠梓區仁昌里', '楠梓區加昌里', '楠梓區和昌里', '楠梓區國昌里', '楠梓區大昌里', '楠梓區宏昌里', '楠梓區宏榮里', '楠梓區宏毅里', '楠梓區廣昌里', '楠梓區建昌里', '楠梓區惠楠里', '楠梓區惠民里', '楠梓區惠豐里', '楠梓區慶昌里', '楠梓區東寧里', '楠梓區泰昌里', '楠梓區清豐里', '楠梓區瑞屏里', '楠梓區福昌里', '楠梓區秀昌里', '楠梓區稔田里', '楠梓區翠屏里', '楠梓區興昌里', '楠梓區藍田里', '楠梓區裕昌里', '楠梓區金田里', '楠梓區錦屏里', '楠梓區隆昌里', '苓雅區中正里', '苓雅區

In [10]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-162f2eaf-eef3-4ec0-94df-77cc5353a9da',
    'IBM_API_KEY_ID': 'TRxbSWfQmx-ZTNpBHWwbZdlCXaHnlFECGoCleCY8USY5',
    'ENDPOINT': 'https://s3.ap-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'courseracapstone-donotdelete-pr-nqeyo6hfku3v0b',
    'FILE': 'CurrentPopByAge Group_2018.csv'
}
body = client_990a7e4b614046be8a53265290fc9cab.get_object(Bucket='courseracapstone-donotdelete-pr-nqeyo6hfku3v0b',Key='CurrentPopByAge Group_2018.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_age_all = pd.read_csv(body)

df_male = df_age_all[df_age_all['性別'] == '男'].drop(index=1,axis=0)
print(df_male.head())

import plotly.express as px
age_fig = px.bar(df_male, x='區別', y='15至39歲', title='Age from 15 to 39 per district.')
age_fig    


     區別 性別      總計   0至6歲   0至12歲   0至14歲  15至39歲  40至64歲  15至64歲   20歲以上  \
4   鹽埕區  男  11,998    463     830     989   3,799   4,835   8,634  10,478   
7   鼓山區  男  67,219  5,006   9,407  10,776  22,207  25,680  47,887  52,797   
10  左營區  男  94,768  6,230  12,364  14,684  32,789  36,912  69,701  73,188   
13  楠梓區  男  90,306  6,343  11,766  13,619  34,577  32,450  67,027  71,286   
16  三民一  男  39,185  1,881   3,347   3,944  12,893  15,049  27,942  33,338   

     65歲以上  70歲以上  100歲以上  
4    2,375  1,452       1  
7    8,556  5,013       3  
10  10,383  6,400       6  
13   9,660  5,535      10  
16   7,299  4,518       1  


In [9]:
age_fig = px.bar(df_male, x='區別', y='40至64歲', title='Age from 40 to 64 per district.')
age_fig

In [11]:
px.bar(df_male, x='區別', y='總計', title='Total')